In [48]:
import pandas as pd
from itertools import repeat
import re

### Read column headers from fantom5 data. 
Read the column headers and extract sample information from it. 

In [1]:
!ls data

fantom5_ds.txt				   samples1829
fantom5_head2000.txt			   samples1829_LIBRARY_IDs
hg19.cage_peak_phase1and2combined_ann.txt  samples1829_simplified
human_samples_nature13182-s2


In [44]:
!grep "^##ColumnVariables" data/fantom5_ds.txt | cut -d"=" -f2 > data/column_vars.txt
col_info = !cat data/column_vars.txt

In [42]:
col_info[:10]

['CAGE peak id',
 'short form of the description below. Common descriptions in the long descriptions has been omited',
 'description of the CAGE peak',
 'transcript which 5end is the nearest to the the CAGE peak',
 'entrezgene (genes) id associated with the transcript',
 'hgnc (gene symbol) id associated with the transcript',
 'uniprot (protein) id associated with the transcript',
 'tpm of 293SLAM rinderpest infection, 00hr, biol_rep1.CNhs14406.13541-145H4',
 'tpm of 293SLAM rinderpest infection, 00hr, biol_rep2.CNhs14407.13542-145H5',
 'tpm of 293SLAM rinderpest infection, 00hr, biol_rep3.CNhs14408.13543-145H6']

In [8]:
sample_info = col_info[7:]

In [75]:
LIB_ID_REGEX = re.compile(r'CNhs(\d+)')
def process_sample_info(info_line):
    cols = [x.strip() for x in re.split(',\s*(?![^()]*\))', info_line)]
    assert 1 <= len(cols) <= 4, "Unknown number of list items: {0}".format(cols)
    col_info = dict(zip(['sample_name', 'sample_id', 'lib_id', 'time', 'donor'], repeat(None)))
    try:
        lib_id = LIB_ID_REGEX.search(info_line).group()
    except AttributeError:
        assert False, "no library id given in info_line"
    col_info["lib_id"] = lib_id
    
    ## reduce four column case to three. 
    if len(cols) == 4: 
        cols[0] == cols[0] + cols[1]
        del cols[1]
    
    ## handle the different header types. 
    if len(cols) == 1:
        col_info['sample_id'] = cols[0]
        col_info['sample_name'] = cols[0].split(".")[0]
    elif len(cols) == 2:
        col_info['sample_name'] = cols[0]
        col_info['sample_id'] = cols[1]
    elif len(cols) == 3: 
        col_info['sample_name'] = cols[0]
        col_info['sample_id'] = cols[1]
        if "hr" in cols[1] or "day" in cols[1]:
            col_info["time"] = cols[1]
        elif "donor" in cols[1]:
            col_info["donor"] = cols[1]
        else:
            assert False, "unknown second column: {0}".format(cols[1])        
    return col_info
    

In [76]:
sample_info_proc = [process_sample_info(info_line) for info_line in sample_info]

AssertionError: unknown second column: expanded and stimulated

In [32]:
pd.DataFrame(sample_info_proc)

,donor,lib_id,sample_name,time
0,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
1,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
2,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
3,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
4,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
5,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
6,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
7,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
8,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
9,None,biol_rep2.CNhs14407.13542-145H5,tpm of 293SLAM rinderpest infection,00hr
